In [4]:
!unzip fine_tuned_with_cs.zip

Archive:  fine_tuned_with_cs.zip
   creating: llama3.2-3b-qlora-summary/
  inflating: llama3.2-3b-qlora-summary/README.md  
   creating: llama3.2-3b-qlora-summary/checkpoint-100/
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/README.md  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/adapter_config.json  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/training_args.bin  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/special_tokens_map.json  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/tokenizer_config.json  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/rng_state.pth  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/trainer_state.json  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/chat_template.jinja  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/optimizer.pt  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/scheduler.pt  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/adapter_model.safetensors  
  infl

In [5]:
!unzip final-summary.zip

Archive:  final-summary.zip
   creating: final-summary/
  inflating: final-summary/README.md  
   creating: final-summary/checkpoint-100/
  inflating: final-summary/checkpoint-100/README.md  
  inflating: final-summary/checkpoint-100/adapter_config.json  
  inflating: final-summary/checkpoint-100/training_args.bin  
  inflating: final-summary/checkpoint-100/special_tokens_map.json  
  inflating: final-summary/checkpoint-100/tokenizer_config.json  
  inflating: final-summary/checkpoint-100/rng_state.pth  
  inflating: final-summary/checkpoint-100/trainer_state.json  
  inflating: final-summary/checkpoint-100/chat_template.jinja  
  inflating: final-summary/checkpoint-100/optimizer.pt  
  inflating: final-summary/checkpoint-100/scheduler.pt  
  inflating: final-summary/checkpoint-100/adapter_model.safetensors  
  inflating: final-summary/checkpoint-100/tokenizer.json  
  inflating: final-summary/adapter_config.json  
  inflating: final-summary/training_args.bin  
   creating: final-summa

In [1]:
!pip install pandas datasets

In [2]:
!pip install transformers torch

In [3]:
!pip install xformers trl peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 139.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [6]:
import json
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
import torch
from gc import collect
from pathlib import Path

In [7]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("We would be using this device:", device)

We would be using this device: cuda


In [8]:
# Load JSONL data (Custom Dataset)
print("\n2-[1/8] Loading dataset...")
data = []
with open("custom_dataset.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

custom_dataset = Dataset.from_dict({
    "messages": [item["messages"] for item in data]
})
split_dataset = custom_dataset.train_test_split(test_size=0.1, seed=42)
val_dataset = split_dataset["test"]

print(f"✓ Loaded {len(val_dataset)} examples")


2-[1/8] Loading dataset...
✓ Loaded 101 examples


In [9]:
def extract_msg_for_base_model(_msg):
    messages = _msg["messages"]
    return {
        "messages": [
            {
                "role": "system",
                "content": """You are an AI assistant that helps provide the summary for a youtube video given we provide you with a user search text, and based on the search text youtube has given out 4 relevant videos as a result, we will give you the search text, video title and the transcript of all the result videos (it could be between 1 to 4 ), your job is to summarize the video according to the prompt and also address the similarities and differences in the video's opinion over the user's query.

for example:

SEARCH_TEXT: Whats the Best thing to do on Holidays

VIDEO_1_TITLE: Top 5 things to do on Holidays
VIDEO_1_TRANSCRIPT: One of the Thing to do on Holiday is to visit your grandparents and spend some time with your family....

VIDEO_2_TITLE: Top 5 Games to try on Holidays
VIDEO_2_TRANSCRIPT: Well you can try Death Stranding or Far Cry (Entire Series) so that u have time to play and also see how they grow over time and you won't see how time passes so quickly...

VIDEO_3_TITLE: How to Convince your boss to work on Holidays
VIDEO_3_TRANSCRIPT: Tells ways to work on Holidays instead of Enjoying...

I am expecting Output like...

Key points from Video_1:
...

Key points from Video_2:
...

Transcript 2 unpacked:
...

well Video 1 and Video 2 agree on the fact that you should spend some time in leisure during holiday be it playing games or with your family. Video 1 seems to be giving good advice and Suggestions from Video 2 can also be considered if you are into games, while Video 3 out of no where suggesting things that might not be average thing to do on Holidays, it focuses more on Working than Enjoying as opposed to  other videos


That's the Example, please make sure to follow these rules as well.
- No bullet points, no lists, no headings.
- Do not use any characters that cannot be typed directly on a standard keyboard, including but not limited to em-dashes (-), en-dashes (-), smart quotes (" ", ' '), or any other special typographic symbols. Use only standard ASCII characters.
- You may also compare videos, for example: "video 1 is similar to video 2 except for this part where they disagree", or "video 3 is more similar to video 1 where the speakers have the same views on this...".
- Combine the relevant points from the transcripts into smooth, continuous paragraph(s) for each video.
- Ignore irrelevant or repeated transcript content.
- Tone should be clear, helpful, and natural.
- Always reference videos by their index like VIDEO_1, VIDEO_2.
- For each video, write a short section summarizing only the key insights. Keep it 3–5 sentences or around 50–100 words.
"""
            },
            {
                "role": "user",
                "content": messages[1]["content"]
            }
        ]
    }

In [10]:
base_generation_inputs = val_dataset.map(extract_msg_for_base_model)

Map:   0%|          | 0/101 [00:00<?, ? examples/s]

In [11]:
BASE_MODEL = "meta-llama/Llama-3.2-3B-Instruct"

In [12]:
bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_compute_dtype=torch.bfloat16,
    )

In [13]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `YTA-DEV` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/hu

# Model_1 Results

Results from the Base Model (LLMA 3.2 3B Instruct)

In [15]:
base_tokenizer = AutoTokenizer.from_pretrained(
        BASE_MODEL,
        use_fast=True
    )
# LLaMA models usually do not have a pad token by default
if base_tokenizer.pad_token is None:
    base_tokenizer.pad_token = base_tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
base_model.eval()
base_outputs = []
for base_msg in base_generation_inputs:
    prompt = base_tokenizer.apply_chat_template(
        base_msg["messages"],
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = base_tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=base_tokenizer.model_max_length,
    ).to(base_model.device)


    with torch.no_grad():
        output = base_model.generate(
            **inputs,
            max_new_tokens=10_000,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            eos_token_id=base_tokenizer.eos_token_id,
            pad_token_id=base_tokenizer.eos_token_id,
        )

    response = base_tokenizer.decode(
        output[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )

    base_outputs.append((base_msg, response))
    print(len(response))


4472
2173
2588
3493
1333
7101
3200
2747
3450
2684
2382
5195
3245
2329
3437
1777
3072
1929
2397
3252
2971
3297
2854
2745
2402
3726
1992
2768
1412
3116
3023
2304
5197
3501
3816
2858
2933
1648
2644
3583
3513
2842
2248
2041
2094
1816
4051
1813
2608
3987
2073
2224
2359
2120
3774
3856
3009
2167
3069
2287
3755
3099
2327
4028
2822
2137
2387
3402
1843
2294
2069
2056
3564
6023
2666
5656
3596
2184
2093
4498
2338
2207
1648
5847
3027
3232
5577


In [ ]:
len(base_outputs)

In [ ]:
Path("base_outputs.json").write_text(json.dumps(base_outputs))

In [ ]:
del base_model
del base_tokenizer
collect()

# Model_2 Results

Results from the Model which was Fine Tuned with only the Custom Dataset

In [14]:
ADAPTER_PATH = "./llama3.2-3b-qlora-summary"

In [15]:
def split_msgs(_msg):
    messages = _msg["messages"]
    return {
        "messages": messages[: 2]
    }

In [16]:
generation_inputs = val_dataset.map(split_msgs)

Map:   0%|          | 0/101 [00:00<?, ? examples/s]

In [17]:
cs_tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH)

cs_tokenizer.pad_token = cs_tokenizer.eos_token

cs_tokenizer.padding_side = "right"

cs_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=bnb_config,  # SAME as original
)

cs_model = prepare_model_for_kbit_training(cs_model, use_gradient_checkpointing=True)
cs_model = PeftModel.from_pretrained(
    cs_model,
    ADAPTER_PATH,
    is_trainable=True   # to continue training
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [19]:
cs_model.eval()
cs_outputs = []
batch_size = 4  # increase if GPU allows
total = len(generation_inputs)

# Cache for speed
tokenizer = cs_tokenizer
apply_template = tokenizer.apply_chat_template
device = cs_model.device

with torch.no_grad():
    for start in range(0, total, batch_size):
        batch = generation_inputs[start:start + batch_size]

        # Apply chat template
        prompts = [
            apply_template(
                msg,
                tokenize=False,
                add_generation_prompt=True
            )
            for msg in batch["messages"]
        ]

        # Tokenize with padding
        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=tokenizer.model_max_length,
        ).to(device)

        # Generate (FAST PATH)
        outputs = cs_model.generate(
            **inputs,
            max_new_tokens=1024,
            do_sample=False,  # GREEDY = FAST
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            use_cache=True,
        )

        input_lens = inputs["input_ids"].shape[1]

        # Decode outputs
        for i, output in enumerate(outputs):
            response = tokenizer.decode(
                output[input_lens:],
                skip_special_tokens=True
            )
            cs_outputs.append((batch["messages"][i], response))

        progress = min(start + batch_size, total) / total * 100
        print(f"{progress:.2f}% complete")


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


3.96% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


7.92% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

87.13% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


91.09% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


95.05% complete
99.01% complete
100.00% complete


In [20]:
Path("cs_outptus.json").write_text(json.dumps(cs_outputs))

2139630

In [21]:
del cs_model
del cs_tokenizer
collect()

12296

# Model_3 Results

Results from the Model which was Fine Tuned with the TL;DR and then Custom Dataset

In [30]:
ADAPTER_PATH = "./final-summary"

In [31]:
cts_tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH)
cts_tokenizer.pad_token = cts_tokenizer.eos_token
cts_tokenizer.padding_side = "right"
cts_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=bnb_config,  # SAME as original
)

cts_model = prepare_model_for_kbit_training(cts_model, use_gradient_checkpointing=True)
cts_model = PeftModel.from_pretrained(
    cts_model,
    ADAPTER_PATH,
    is_trainable=True   # to continue training
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [32]:
cts_model.eval()
cts_outputs = []
batch_size = 6  # increase if GPU allows
total = len(generation_inputs)

# Cache for speed
tokenizer = cts_tokenizer
apply_template = tokenizer.apply_chat_template
device = cts_model.device

with torch.no_grad():
    for start in range(0, total, batch_size):
        batch = generation_inputs[start:start + batch_size]

        # Apply chat template
        prompts = [
            apply_template(
                msg,
                tokenize=False,
                add_generation_prompt=True
            )
            for msg in batch["messages"]
        ]

        # Tokenize with padding
        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=tokenizer.model_max_length,
        ).to(device)

        # Generate (FAST PATH)
        outputs = cts_model.generate(
            **inputs,
            max_new_tokens=1024,
            do_sample=False,  # GREEDY = FAST
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            use_cache=True,
        )

        input_lens = inputs["input_ids"].shape[1]

        # Decode outputs
        for i, output in enumerate(outputs):
            response = tokenizer.decode(
                output[input_lens:],
                skip_special_tokens=True
            )
            cts_outputs.append((batch["messages"][i], response))

        progress = min(start + batch_size, total) / total * 100
        print(f"{progress:.2f}% complete")




A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


5.94% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


11.88% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

83.17% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


89.11% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


95.05% complete
100.00% complete


In [33]:
Path("cts_outputs.json").write_text(json.dumps(cts_outputs))

2208586